In [1]:
import tensorflow as tf
import keras
from keras import layers

2023-08-21 16:36:39.298828: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-21 16:36:39.299921: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 16:36:39.324737: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 16:36:39.325333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 16:36:39.931665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [47]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"), # After this step the matrix is 3x2
        layers.Dense(3, activation="relu", name="layer2"), # After this step the matrix is 3x3 (3x2 * 2x3)
        layers.Dense(4, name="layer3"), # After this step the matrix is 3x4 (3x3 * 3x4)
    ]
)

x = tf.ones((3, 3))
y = model(x)
#print(y.numpy())
for layer in model.layers:
    print(layer.name)
    weights, biases = layer.get_weights()
    print("Weights:", weights)
    print("Biases:", biases)
    print("------------")


layer1
Weights: [[-1.0493544   0.7363399 ]
 [-0.19919515  0.207973  ]
 [-0.18664336 -0.84004706]]
Biases: [0. 0.]
------------
layer2
Weights: [[-0.21164376 -0.7095196  -0.45058084]
 [-0.7338321   0.1422962  -0.51181006]]
Biases: [0. 0. 0.]
------------
layer3
Weights: [[-0.369933   -0.89441603  0.08582222 -0.8300144 ]
 [ 0.9001694   0.08848596  0.32949078  0.28434944]
 [ 0.23469234  0.629735    0.3881148   0.35783112]]
Biases: [0. 0. 0. 0.]
------------
